# Fed-BioMed Researcher base example

This example uses MNIST dataset. Please check `README.md` file in `notebooks` directory for the instructions to load MNIST dataset and configure nodes.


## Define an experiment model and parameters

Declare a torch training plan MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    
    # Defines and return model 
    def init_model(self, model_args):
        return self.Net(model_args = model_args)
    
    # Defines and return optimizer
    def init_optimizer(self, optimizer_args):
        return torch.optim.Adam(self.model().parameters(), lr = optimizer_args["lr"])
    
    # Declares and return dependencies
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms"]
        return deps
    
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = nn.Dropout(0.25)
            self.dropout2 = nn.Dropout(0.5)
            self.fc1 = nn.Linear(9216, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = F.relu(x)
            x = self.conv2(x)
            x = F.relu(x)
            x = F.max_pool2d(x, 2)
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = F.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)


            output = F.log_softmax(x, dim=1)
            return output

    def training_data(self):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.model().forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


### Model arguments and training arguments

In [2]:
model_args = {}

training_args = {
    'loader_args': { 'batch_size': 48, }, 
    'optimizer_args': {
        "lr" : 1e-3
    },
    # 'use_gpu': True,  # automatically falls back to cpu on nodes that don't support gpu
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

### Create and run the experiment

In [5]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage
from fedbiomed.researcher.secagg import SecureAggregationSchemes, SecureAggregation


tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage()
                )

2024-09-03 17:10:53,080 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2024-09-03 17:10:53,088 fedbiomed DEBUG - Node: NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f polling for the tasks

2024-09-03 17:10:53,090 fedbiomed DEBUG - Node: NODE_16ecdabb-f02b-4973-94f8-a9f9e5c04412 polling for the tasks

2024-09-03 17:10:53,094 fedbiomed INFO - Node selected for training -> NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f

2024-09-03 17:10:53,094 fedbiomed INFO - Node selected for training -> NODE_16ecdabb-f02b-4973-94f8-a9f9e5c04412

2024-09-03 17:10:53,100 fedbiomed DEBUG - Model file has been saved: /Users/scansiz/development/fedbiomed/var/experiments/Experiment_0005/model_e6fac7ad-2c31-49ad-b8ea-398e7985e3b2.py

In [ ]:
exp.info()

In [7]:
exp.run(rounds=4, increase=True)

2024-09-03 17:11:21,943 fedbiomed DEBUG - Auto increasing total rounds for experiment from 2 to 6

2024-09-03 17:11:21,948 fedbiomed INFO - Sampled nodes in round 2 ['NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f', 'NODE_16ecdabb-f02b-4973-94f8-a9f9e5c04412']

2024-09-03 17:11:21,955 fedbiomed INFO - Sending request 
					 To: NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-09-03 17:11:21,956 fedbiomed INFO - Sending request 
					 To: NODE_16ecdabb-f02b-4973-94f8-a9f9e5c04412 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-09-03 17:11:21,993 fedbiomed DEBUG - Node: NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f polling for the tasks

...

2024-09-03 17:11:37,136 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f 
					 Round 6 Epoch: 1 | Iteration: 90/100 (90%) | Samples: 4320/4800
 					 Loss: 0.169790 
					 ---------

2024-09-03 17:11:37,486 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_16ecdabb-f02b-4973-94f8-a9f9e5c04412 
					 Round 6 Epoch: 1 | Iteration: 100/100 (100%) | Samples: 4800/4800
 					 Loss: 0.075609 
					 ---------

2024-09-03 17:11:37,511 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f 
					 Round 6 Epoch: 1 | Iteration: 100/100 (100%) | Samples: 4800/4800
 					 Loss: 0.089010 
					 ---------

2024-09-03 17:11:37,531 fedbiomed INFO - Nodes that successfully reply in round 5 ['NODE_c6864947-8a25-4b11-aa49-fe9f41a4d98f', 'NODE_16ecdabb-f02b-4973-94f8-a9f9e5c04412']

4

In [ ]:
exp.run_once(increase=True)

Save trained model to file

In [ ]:
exp.training_plan().export_model('./trained_model')

#### Display results

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())
